# SMOTE-based Anomaly Traffic Generation

This notebook generates synthetic anomaly BGP traffic using SMOTE-based oversampling techniques.
It filters data with high confidence anomaly labels: `medium_confidence`, `high_confidence`, `very_high_confidence`.

## 1. Setup and Installation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from scipy import stats
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully!")

## 2. Define Feature Sets with Enhanced Categorization

In [ ]:
# High confidence anomaly labels to filter
HIGH_CONFIDENCE_LABELS = ['medium_confidence', 'high_confidence', 'very_high_confidence']

# Integer features (must be rounded after SMOTE)
INTEGER_FEATURES = [
    'announcements', 'withdrawals', 'nlri_ann', 'dups',
    'origin_0', 'origin_2', 'origin_changes',
    'imp_wd', 'imp_wd_spath', 'imp_wd_dpath',
    'as_path_max', 'unique_as_path_max',
    'edit_distance_max',
    'edit_distance_dict_0', 'edit_distance_dict_1', 'edit_distance_dict_2',
    'edit_distance_dict_3', 'edit_distance_dict_4', 'edit_distance_dict_5',
    'edit_distance_dict_6',
    'edit_distance_unique_dict_0', 'edit_distance_unique_dict_1',
    'number_rare_ases',
    'nadas', 'flaps'
]

# Heavy-tailed count features (benefit from log1p transform before SMOTE)
HEAVY_TAILED_FEATURES = [
    'announcements', 'withdrawals', 'nlri_ann',
    'number_rare_ases', 'nadas'
]

# Continuous features (can be float but bounded)
CONTINUOUS_FEATURES = [
    'edit_distance_avg',
    'rare_ases_avg'
]

# Bounded ratio features (need empirical min/max clipping)
BOUNDED_RATIO_FEATURES = [
    'rare_ases_avg'  # This is a ratio typically between 0 and 1
]

# Edit distance dict features (for constraint validation)
EDIT_DISTANCE_DICT_FEATURES = [
    'edit_distance_dict_0', 'edit_distance_dict_1', 'edit_distance_dict_2',
    'edit_distance_dict_3', 'edit_distance_dict_4', 'edit_distance_dict_5',
    'edit_distance_dict_6'
]

# Core features for quality validation
CORE_VALIDATION_FEATURES = [
    'announcements', 'withdrawals', 'nlri_ann', 'edit_distance_avg'
]

# All features to use for synthetic generation
ALL_FEATURES = INTEGER_FEATURES + CONTINUOUS_FEATURES

# Columns to exclude (labels, timestamps, derived scores)
EXCLUDE_COLS = [
    'label', 'confidence_label', 'window_start', 'window_end',
    'iso_forest_score', 'lof_score', 'statistical_score',
    'elliptic_score', 'cluster', 'anomaly_votes', 'consensus_score',
    'Incident'
]

# Potential grouping columns (if available in dataset)
POTENTIAL_GROUP_COLS = ['peer_asn', 'peer_ip', 'prefix', 'collector', 'Incident']

print(f"Integer features: {len(INTEGER_FEATURES)}")
print(f"Heavy-tailed features: {len(HEAVY_TAILED_FEATURES)}")
print(f"Continuous features: {len(CONTINUOUS_FEATURES)}")
print(f"Total features for generation: {len(ALL_FEATURES)}")

## 3. Load and Explore Data

In [ ]:
# Load data - Anomaly data with reinforced labels
DATA_PATH = '/home/smotaali/BGP_Traffic_Generation/RIPE/RIPE_INCIDENTS/all_incidents_anomalies_reinforced_v2.csv'

df = pd.read_csv(DATA_PATH)
print(f"Original dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

In [ ]:
# Check label distributions
print("Confidence label distribution:")
print(df['confidence_label'].value_counts())

if 'label' in df.columns:
    print("\nOriginal label distribution:")
    print(df['label'].value_counts())

In [ ]:
# Filter only high confidence anomaly samples
df_high_confidence = df[df['confidence_label'].isin(HIGH_CONFIDENCE_LABELS)].copy()
print(f"High confidence anomaly samples: {len(df_high_confidence)}")
print(f"\nDistribution within high confidence:")
print(df_high_confidence['confidence_label'].value_counts())

In [ ]:
# Verify all features exist in the dataset
missing_features = [f for f in ALL_FEATURES if f not in df.columns]
if missing_features:
    print(f"WARNING: Missing features: {missing_features}")
    # Update ALL_FEATURES to only include available features
    ALL_FEATURES = [f for f in ALL_FEATURES if f in df.columns]
    INTEGER_FEATURES = [f for f in INTEGER_FEATURES if f in df.columns]
    HEAVY_TAILED_FEATURES = [f for f in HEAVY_TAILED_FEATURES if f in df.columns]
    CONTINUOUS_FEATURES = [f for f in CONTINUOUS_FEATURES if f in df.columns]
    print(f"Updated to {len(ALL_FEATURES)} available features")
else:
    print("All features found in dataset!")

In [ ]:
# Check for potential grouping columns
available_group_cols = [col for col in POTENTIAL_GROUP_COLS if col in df.columns]
print(f"Available grouping columns: {available_group_cols}")

if available_group_cols:
    for col in available_group_cols:
        n_unique = df_high_confidence[col].nunique()
        print(f"  {col}: {n_unique} unique values")

In [ ]:
# Prepare features - use only raw BGP features
X_anomaly = df_high_confidence[ALL_FEATURES].copy()

# Handle any missing values
X_anomaly = X_anomaly.fillna(X_anomaly.median())

print(f"Feature matrix shape: {X_anomaly.shape}")
X_anomaly.describe()

In [ ]:
# Store empirical bounds for all features (for clipping after inverse transform)
EMPIRICAL_BOUNDS = {}
for col in ALL_FEATURES:
    EMPIRICAL_BOUNDS[col] = {
        'min': X_anomaly[col].min(),
        'max': X_anomaly[col].max(),
        'q01': X_anomaly[col].quantile(0.01),
        'q99': X_anomaly[col].quantile(0.99)
    }

print("Empirical bounds computed for all features.")
print("\nExample bounds for key features:")
for feat in ['announcements', 'withdrawals', 'rare_ases_avg', 'edit_distance_avg']:
    if feat in EMPIRICAL_BOUNDS:
        print(f"  {feat}: [{EMPIRICAL_BOUNDS[feat]['min']:.4f}, {EMPIRICAL_BOUNDS[feat]['max']:.4f}]")

## 4. Enhanced Preprocessing: Log1p Transform for Heavy-Tailed Features

In [ ]:
def apply_log1p_transform(df, features):
    """
    Apply log1p transform to heavy-tailed features.
    
    This helps SMOTE work better with count data that has high skewness.
    log1p(x) = log(1 + x) handles zeros gracefully.
    
    Returns:
        Transformed dataframe and dict of original stats for monitoring
    """
    result = df.copy()
    original_stats = {}
    
    for col in features:
        if col in result.columns:
            original_stats[col] = {
                'mean': result[col].mean(),
                'std': result[col].std(),
                'skew': result[col].skew(),
                'max': result[col].max()
            }
            result[col] = np.log1p(result[col])
            
    return result, original_stats


def apply_expm1_inverse(df, features):
    """
    Apply inverse log1p transform (expm1).
    expm1(x) = exp(x) - 1
    """
    result = df.copy()
    
    for col in features:
        if col in result.columns:
            result[col] = np.expm1(result[col])
            # Ensure non-negative after inverse transform
            result[col] = np.maximum(result[col], 0)
            
    return result


# Test transform on sample data
print("Testing log1p transform on heavy-tailed features:")
print("="*50)

for col in HEAVY_TAILED_FEATURES:
    if col in X_anomaly.columns:
        original_skew = X_anomaly[col].skew()
        transformed_skew = np.log1p(X_anomaly[col]).skew()
        print(f"{col}:")
        print(f"  Original skewness: {original_skew:.2f}")
        print(f"  Transformed skewness: {transformed_skew:.2f}")

## 5. Enhanced Post-Processing with BGP Semantic Constraints

In [ ]:
def post_process_synthetic(df, empirical_bounds=None, enforce_bgp_constraints=True):
    """
    Enhanced post-processing for synthetic data with:
    1. Integer rounding and non-negative constraints
    2. Empirical min/max clipping (not just >= 0)
    3. BGP semantic constraints enforcement
    
    Parameters:
    -----------
    df : pd.DataFrame
        Synthetic data to post-process
    empirical_bounds : dict
        Dictionary with min/max bounds for each feature
    enforce_bgp_constraints : bool
        Whether to enforce BGP domain-specific constraints
    """
    result = df.copy()
    
    # Step 1: Round integer features and ensure non-negative
    for col in INTEGER_FEATURES:
        if col in result.columns:
            result[col] = result[col].round().astype(int).clip(lower=0)
    
    # Step 2: Ensure continuous features are non-negative
    for col in CONTINUOUS_FEATURES:
        if col in result.columns:
            result[col] = result[col].clip(lower=0)
    
    # Step 3: Apply empirical bounds clipping (especially for bounded ratios)
    if empirical_bounds is not None:
        for col in BOUNDED_RATIO_FEATURES:
            if col in result.columns and col in empirical_bounds:
                bounds = empirical_bounds[col]
                result[col] = result[col].clip(
                    lower=bounds['min'],
                    upper=bounds['max']
                )
    
    # Step 4: Enforce BGP semantic constraints
    if enforce_bgp_constraints:
        result = enforce_bgp_semantic_constraints(result)
    
    return result


def enforce_bgp_semantic_constraints(df):
    """
    Enforce BGP-specific structural constraints on synthetic data.
    
    Constraints:
    1. nlri_ann <= announcements (NLRI count cannot exceed announcements)
    2. origin_0 + origin_2 <= announcements (origin types are subset of announcements)
    3. imp_wd_spath + imp_wd_dpath <= imp_wd (implicit withdrawals breakdown)
    4. edit_distance_dict sums should be consistent with edit_distance_max
    5. dups <= announcements (duplicates cannot exceed announcements)
    """
    result = df.copy()
    n_violations = {}
    
    # Constraint 1: nlri_ann <= announcements
    if 'nlri_ann' in result.columns and 'announcements' in result.columns:
        mask = result['nlri_ann'] > result['announcements']
        n_violations['nlri_ann > announcements'] = mask.sum()
        result.loc[mask, 'nlri_ann'] = result.loc[mask, 'announcements']
    
    # Constraint 2: origin_0 + origin_2 <= announcements
    if all(col in result.columns for col in ['origin_0', 'origin_2', 'announcements']):
        origin_sum = result['origin_0'] + result['origin_2']
        mask = origin_sum > result['announcements']
        n_violations['origin_0 + origin_2 > announcements'] = mask.sum()
        if mask.any():
            # Scale down proportionally
            scale_factor = result.loc[mask, 'announcements'] / origin_sum[mask]
            result.loc[mask, 'origin_0'] = (result.loc[mask, 'origin_0'] * scale_factor).round().astype(int)
            result.loc[mask, 'origin_2'] = (result.loc[mask, 'origin_2'] * scale_factor).round().astype(int)
    
    # Constraint 3: imp_wd_spath + imp_wd_dpath <= imp_wd
    if all(col in result.columns for col in ['imp_wd_spath', 'imp_wd_dpath', 'imp_wd']):
        imp_sum = result['imp_wd_spath'] + result['imp_wd_dpath']
        mask = imp_sum > result['imp_wd']
        n_violations['imp_wd_spath + imp_wd_dpath > imp_wd'] = mask.sum()
        if mask.any():
            scale_factor = result.loc[mask, 'imp_wd'] / imp_sum[mask].replace(0, 1)
            result.loc[mask, 'imp_wd_spath'] = (result.loc[mask, 'imp_wd_spath'] * scale_factor).round().astype(int)
            result.loc[mask, 'imp_wd_dpath'] = (result.loc[mask, 'imp_wd_dpath'] * scale_factor).round().astype(int)
    
    # Constraint 4: dups <= announcements
    if 'dups' in result.columns and 'announcements' in result.columns:
        mask = result['dups'] > result['announcements']
        n_violations['dups > announcements'] = mask.sum()
        result.loc[mask, 'dups'] = result.loc[mask, 'announcements']
    
    # Constraint 5: edit_distance_dict values should have max <= edit_distance_max
    if 'edit_distance_max' in result.columns:
        for ed_col in EDIT_DISTANCE_DICT_FEATURES:
            if ed_col in result.columns:
                # Individual edit distances shouldn't exceed max
                # (This is a soft constraint - edit_distance_dict_i represents count at distance i)
                pass  # This constraint is more complex and domain-specific
    
    # Report violations fixed
    total_violations = sum(n_violations.values())
    if total_violations > 0:
        print(f"\nBGP constraint violations fixed: {total_violations}")
        for constraint, count in n_violations.items():
            if count > 0:
                print(f"  {constraint}: {count}")
    
    return result


print("Post-processing functions defined!")

## 6. k_neighbors Validation and Safe SMOTE Configuration

In [ ]:
def validate_k_neighbors(n_samples, requested_k, method_name="SMOTE"):
    """
    Validate and adjust k_neighbors to avoid silent failures.
    
    SMOTE requires k_neighbors < number of minority samples.
    This function ensures safe k_neighbors selection and warns if adjusted.
    
    Parameters:
    -----------
    n_samples : int
        Number of samples in the minority class
    requested_k : int
        Requested number of neighbors
    method_name : str
        Name of the method for logging
        
    Returns:
    --------
    int : Safe k_neighbors value
    """
    max_k = n_samples - 1
    
    if requested_k >= n_samples:
        safe_k = max(1, max_k)
        print(f"WARNING [{method_name}]: k_neighbors ({requested_k}) >= n_samples ({n_samples})")
        print(f"  Adjusting k_neighbors to {safe_k}")
        return safe_k
    
    return requested_k


def get_safe_smote_config(n_minority_samples, base_k=5):
    """
    Get safe SMOTE configuration based on available samples.
    
    Returns a dictionary with recommended parameters.
    """
    config = {
        'k_neighbors': validate_k_neighbors(n_minority_samples, base_k),
        'n_minority_samples': n_minority_samples,
        'is_safe': n_minority_samples >= 2  # Minimum for SMOTE
    }
    
    if not config['is_safe']:
        print(f"WARNING: Not enough minority samples ({n_minority_samples}) for SMOTE. Minimum is 2.")
    
    return config


# Test validation
print("Testing k_neighbors validation:")
print("="*50)
test_cases = [(100, 5), (10, 5), (5, 5), (3, 5), (2, 5)]
for n_samples, requested_k in test_cases:
    safe_k = validate_k_neighbors(n_samples, requested_k, "Test")
    print(f"  n_samples={n_samples}, requested_k={requested_k} -> safe_k={safe_k}")

## 7. Enhanced SMOTE Generation Functions

In [ ]:
def generate_with_smote_enhanced(
    X, 
    n_samples, 
    use_log_transform=True,
    enforce_constraints=True,
    random_state=42
):
    """
    Enhanced SMOTE generation with:
    - Log1p transform for heavy-tailed features
    - k_neighbors validation
    - Empirical bounds clipping
    - BGP constraint enforcement
    """
    n_original = len(X)
    
    # Step 1: Apply log1p transform if requested
    if use_log_transform:
        X_work, _ = apply_log1p_transform(X.copy(), HEAVY_TAILED_FEATURES)
    else:
        X_work = X.copy()
    
    # Step 2: Create artificial labels with validated minority size
    minority_size = max(10, int(n_original * 0.01))
    
    # Validate k_neighbors
    safe_k = validate_k_neighbors(minority_size, 5, "Standard SMOTE")
    
    np.random.seed(random_state)
    minority_idx = np.random.choice(n_original, minority_size, replace=False)
    
    X_array = X_work.values
    y = np.zeros(n_original)
    y[minority_idx] = 1
    
    desired_minority = n_samples + minority_size
    
    # Step 3: Apply SMOTE with safe k_neighbors
    smote = SMOTE(
        sampling_strategy={1: desired_minority},
        k_neighbors=safe_k,
        random_state=random_state
    )
    
    X_resampled, y_resampled = smote.fit_resample(X_array, y)
    synthetic_samples = X_resampled[y_resampled == 1][minority_size:]
    
    result = pd.DataFrame(synthetic_samples, columns=X.columns)
    
    # Step 4: Apply inverse log transform if used
    if use_log_transform:
        result = apply_expm1_inverse(result, HEAVY_TAILED_FEATURES)
    
    # Step 5: Post-process with empirical bounds and BGP constraints
    result = post_process_synthetic(
        result, 
        empirical_bounds=EMPIRICAL_BOUNDS,
        enforce_bgp_constraints=enforce_constraints
    )
    
    return result

In [ ]:
def generate_with_borderline_smote_enhanced(
    X, 
    n_samples, 
    use_log_transform=True,
    enforce_constraints=True,
    kind='borderline-1',  # 'borderline-1' or 'borderline-2'
    random_state=42
):
    """
    Enhanced Borderline SMOTE generation.
    
    BorderlineSMOTE focuses on samples near the decision boundary,
    which may be useful for model training (not necessarily for traffic simulation).
    
    Parameters:
    -----------
    kind : str
        'borderline-1': Only oversample borderline samples
        'borderline-2': Also consider samples from majority class
    """
    n_original = len(X)
    
    if use_log_transform:
        X_work, _ = apply_log1p_transform(X.copy(), HEAVY_TAILED_FEATURES)
    else:
        X_work = X.copy()
    
    minority_size = max(10, int(n_original * 0.01))
    safe_k = validate_k_neighbors(minority_size, 5, "Borderline SMOTE")
    
    np.random.seed(random_state)
    minority_idx = np.random.choice(n_original, minority_size, replace=False)
    
    X_array = X_work.values
    y = np.zeros(n_original)
    y[minority_idx] = 1
    
    desired_minority = n_samples + minority_size
    
    borderline_smote = BorderlineSMOTE(
        sampling_strategy={1: desired_minority},
        k_neighbors=safe_k,
        m_neighbors=min(10, n_original - 1),
        kind=kind,
        random_state=random_state
    )
    
    X_resampled, y_resampled = borderline_smote.fit_resample(X_array, y)
    synthetic_samples = X_resampled[y_resampled == 1][minority_size:]
    
    result = pd.DataFrame(synthetic_samples, columns=X.columns)
    
    if use_log_transform:
        result = apply_expm1_inverse(result, HEAVY_TAILED_FEATURES)
    
    result = post_process_synthetic(
        result,
        empirical_bounds=EMPIRICAL_BOUNDS,
        enforce_bgp_constraints=enforce_constraints
    )
    
    return result

In [ ]:
def generate_with_adasyn_enhanced(
    X, 
    n_samples, 
    use_log_transform=True,
    enforce_constraints=True,
    random_state=42
):
    """
    Enhanced ADASYN generation.
    Adaptively generates more samples in regions where density is low.
    """
    n_original = len(X)
    
    if use_log_transform:
        X_work, _ = apply_log1p_transform(X.copy(), HEAVY_TAILED_FEATURES)
    else:
        X_work = X.copy()
    
    minority_size = max(10, int(n_original * 0.01))
    safe_k = validate_k_neighbors(minority_size, 5, "ADASYN")
    
    np.random.seed(random_state)
    minority_idx = np.random.choice(n_original, minority_size, replace=False)
    
    X_array = X_work.values
    y = np.zeros(n_original)
    y[minority_idx] = 1
    
    desired_minority = n_samples + minority_size
    
    adasyn = ADASYN(
        sampling_strategy={1: desired_minority},
        n_neighbors=safe_k,
        random_state=random_state
    )
    
    try:
        X_resampled, y_resampled = adasyn.fit_resample(X_array, y)
        synthetic_samples = X_resampled[y_resampled == 1][minority_size:]
        result = pd.DataFrame(synthetic_samples, columns=X.columns)
        
        if use_log_transform:
            result = apply_expm1_inverse(result, HEAVY_TAILED_FEATURES)
        
        result = post_process_synthetic(
            result,
            empirical_bounds=EMPIRICAL_BOUNDS,
            enforce_bgp_constraints=enforce_constraints
        )
        return result
    
    except ValueError as e:
        print(f"ADASYN failed: {e}. Falling back to standard SMOTE.")
        return generate_with_smote_enhanced(
            X, n_samples, use_log_transform, enforce_constraints, random_state
        )

In [ ]:
def generate_with_kmeans_smote_enhanced(
    X, 
    n_samples, 
    n_clusters=10, 
    use_log_transform=True,
    enforce_constraints=True,
    random_state=42
):
    """
    Enhanced K-Means clustering + SMOTE.
    Clusters the data first, then applies SMOTE within each cluster.
    """
    np.random.seed(random_state)
    
    if use_log_transform:
        X_work, _ = apply_log1p_transform(X.copy(), HEAVY_TAILED_FEATURES)
    else:
        X_work = X.copy()
    
    X_array = X_work.values
    
    # Scale for clustering
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_array)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)
    
    cluster_sizes = np.bincount(cluster_labels)
    samples_per_cluster = (cluster_sizes / cluster_sizes.sum() * n_samples).astype(int)
    samples_per_cluster = np.maximum(samples_per_cluster, 1)
    
    synthetic_all = []
    
    for cluster_id in range(n_clusters):
        cluster_mask = cluster_labels == cluster_id
        X_cluster = X_array[cluster_mask]
        
        if len(X_cluster) < 3:
            continue
            
        n_to_generate = samples_per_cluster[cluster_id]
        minority_size = max(2, int(len(X_cluster) * 0.1))
        
        # Validate k_neighbors for this cluster
        safe_k = validate_k_neighbors(
            minority_size, 3, f"K-Means Cluster {cluster_id}"
        )
        
        if safe_k < 1:
            continue
            
        minority_idx = np.random.choice(len(X_cluster), minority_size, replace=False)
        
        y_cluster = np.zeros(len(X_cluster))
        y_cluster[minority_idx] = 1
        
        try:
            smote = SMOTE(
                sampling_strategy={1: n_to_generate + minority_size},
                k_neighbors=safe_k,
                random_state=random_state
            )
            X_res, y_res = smote.fit_resample(X_cluster, y_cluster)
            synthetic = X_res[y_res == 1][minority_size:]
            synthetic_all.append(synthetic)
        except Exception as e:
            print(f"  Cluster {cluster_id} failed: {e}")
            continue
    
    if synthetic_all:
        synthetic_combined = np.vstack(synthetic_all)
        result = pd.DataFrame(synthetic_combined[:n_samples], columns=X.columns)
        
        if use_log_transform:
            result = apply_expm1_inverse(result, HEAVY_TAILED_FEATURES)
        
        result = post_process_synthetic(
            result,
            empirical_bounds=EMPIRICAL_BOUNDS,
            enforce_bgp_constraints=enforce_constraints
        )
        return result
    else:
        print("K-Means SMOTE failed for all clusters. Falling back to standard SMOTE.")
        return generate_with_smote_enhanced(
            X, n_samples, use_log_transform, enforce_constraints, random_state
        )

## 8. Stratified SMOTE (Group-based Generation)

In [ ]:
def generate_stratified_smote(
    df_full,
    feature_cols,
    group_col,
    n_samples,
    min_group_size=20,
    use_log_transform=True,
    enforce_constraints=True,
    random_state=42
):
    """
    Stratified SMOTE: Apply SMOTE within subpopulations (e.g., by peer or prefix).
    
    This avoids mixing very different behaviors from different groups.
    
    Parameters:
    -----------
    df_full : pd.DataFrame
        Full dataframe with features and grouping column
    feature_cols : list
        List of feature column names
    group_col : str
        Column to group by (e.g., 'peer_asn', 'prefix', 'Incident')
    n_samples : int
        Total number of samples to generate
    min_group_size : int
        Minimum group size to apply SMOTE (smaller groups are skipped)
    """
    np.random.seed(random_state)
    
    # Get group sizes
    group_sizes = df_full.groupby(group_col).size()
    valid_groups = group_sizes[group_sizes >= min_group_size].index.tolist()
    
    print(f"Stratified SMOTE by '{group_col}':")
    print(f"  Total groups: {len(group_sizes)}")
    print(f"  Valid groups (>= {min_group_size} samples): {len(valid_groups)}")
    
    if len(valid_groups) == 0:
        print(f"  No valid groups found. Falling back to non-stratified SMOTE.")
        return generate_with_smote_enhanced(
            df_full[feature_cols], n_samples, use_log_transform, 
            enforce_constraints, random_state
        )
    
    # Calculate samples per group proportionally
    valid_sizes = group_sizes[valid_groups]
    samples_per_group = (valid_sizes / valid_sizes.sum() * n_samples).astype(int)
    samples_per_group = np.maximum(samples_per_group, 1)
    
    synthetic_all = []
    
    for group_id in valid_groups:
        group_df = df_full[df_full[group_col] == group_id]
        X_group = group_df[feature_cols].copy()
        n_to_generate = samples_per_group[group_id]
        
        if len(X_group) < 10:
            continue
        
        try:
            synthetic_group = generate_with_smote_enhanced(
                X_group, 
                n_to_generate,
                use_log_transform=use_log_transform,
                enforce_constraints=enforce_constraints,
                random_state=random_state + hash(str(group_id)) % 1000
            )
            synthetic_group[group_col] = group_id
            synthetic_all.append(synthetic_group)
        except Exception as e:
            print(f"  Group '{group_id}' failed: {e}")
            continue
    
    if synthetic_all:
        result = pd.concat(synthetic_all, ignore_index=True)
        print(f"  Generated {len(result)} samples across {len(synthetic_all)} groups")
        return result
    else:
        print("  All groups failed. Falling back to non-stratified SMOTE.")
        return generate_with_smote_enhanced(
            df_full[feature_cols], n_samples, use_log_transform,
            enforce_constraints, random_state
        )


print("Stratified SMOTE function defined!")

## 9. Distributional Validation Tests

In [ ]:
def kolmogorov_smirnov_tests(original, synthetic, features=None, alpha=0.05):
    """
    Perform Kolmogorov-Smirnov tests to compare distributions.
    
    The KS test measures the maximum difference between the empirical
    cumulative distribution functions of two samples.
    
    Parameters:
    -----------
    original : pd.DataFrame
        Original data
    synthetic : pd.DataFrame
        Synthetic data
    features : list
        Features to test (default: core validation features)
    alpha : float
        Significance level
        
    Returns:
    --------
    pd.DataFrame : Results with KS statistic and p-value for each feature
    """
    if features is None:
        features = CORE_VALIDATION_FEATURES
    
    results = []
    
    for col in features:
        if col in original.columns and col in synthetic.columns:
            ks_stat, p_value = ks_2samp(original[col], synthetic[col])
            
            results.append({
                'Feature': col,
                'KS Statistic': ks_stat,
                'P-Value': p_value,
                'Significant Diff': 'Yes' if p_value < alpha else 'No',
                'Quality': 'Good' if ks_stat < 0.1 else ('Fair' if ks_stat < 0.2 else 'Poor')
            })
    
    return pd.DataFrame(results)


def compare_correlation_matrices(original, synthetic, features=None):
    """
    Compare correlation matrices between original and synthetic data.
    
    This checks if SMOTE preserves the dependency structure, not just marginals.
    
    Returns:
    --------
    dict with correlation matrices and difference metrics
    """
    if features is None:
        features = CORE_VALIDATION_FEATURES + ['origin_0', 'origin_2', 'dups']
    
    # Ensure features exist in both
    common_features = [f for f in features if f in original.columns and f in synthetic.columns]
    
    corr_original = original[common_features].corr()
    corr_synthetic = synthetic[common_features].corr()
    
    # Calculate difference metrics
    corr_diff = np.abs(corr_original - corr_synthetic)
    
    # Frobenius norm of difference (overall measure)
    frobenius_norm = np.sqrt((corr_diff ** 2).sum().sum())
    
    # Max absolute difference
    max_diff = corr_diff.values.max()
    
    # Mean absolute difference
    mean_diff = corr_diff.values.mean()
    
    return {
        'corr_original': corr_original,
        'corr_synthetic': corr_synthetic,
        'corr_diff': corr_diff,
        'frobenius_norm': frobenius_norm,
        'max_diff': max_diff,
        'mean_diff': mean_diff,
        'features': common_features
    }


def plot_correlation_comparison(corr_results, figsize=(16, 5)):
    """
    Plot correlation matrix comparison.
    """
    fig, axes = plt.subplots(1, 3, figsize=figsize)
    
    # Original correlation
    sns.heatmap(
        corr_results['corr_original'], 
        annot=True, fmt='.2f', cmap='coolwarm',
        center=0, vmin=-1, vmax=1,
        ax=axes[0]
    )
    axes[0].set_title('Original Anomaly Data Correlation')
    
    # Synthetic correlation
    sns.heatmap(
        corr_results['corr_synthetic'], 
        annot=True, fmt='.2f', cmap='coolwarm',
        center=0, vmin=-1, vmax=1,
        ax=axes[1]
    )
    axes[1].set_title('Synthetic Anomaly Data Correlation')
    
    # Difference
    sns.heatmap(
        corr_results['corr_diff'], 
        annot=True, fmt='.2f', cmap='Reds',
        vmin=0, vmax=0.5,
        ax=axes[2]
    )
    axes[2].set_title(f"Absolute Difference (Mean: {corr_results['mean_diff']:.3f})")
    
    plt.tight_layout()
    return fig


def plot_distribution_comparison(original, synthetic, features=None, figsize=(15, 10)):
    """
    Plot distribution comparisons for key features.
    """
    if features is None:
        features = CORE_VALIDATION_FEATURES
    
    n_features = len(features)
    n_cols = 2
    n_rows = (n_features + 1) // 2
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    axes = axes.flatten()
    
    for i, col in enumerate(features):
        ax = axes[i]
        
        # Plot histograms
        ax.hist(original[col], bins=50, alpha=0.5, label='Original', density=True)
        ax.hist(synthetic[col], bins=50, alpha=0.5, label='Synthetic', density=True)
        
        # Add KS statistic
        ks_stat, p_value = ks_2samp(original[col], synthetic[col])
        ax.set_title(f'{col}\nKS={ks_stat:.3f}, p={p_value:.3e}')
        ax.legend()
    
    # Hide unused axes
    for i in range(n_features, len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    return fig


print("Distributional validation functions defined!")

## 10. Enhanced Statistics Comparison

In [ ]:
def compare_statistics_enhanced(original, synthetic, name):
    """
    Enhanced comparison with more metrics and all features.
    """
    print(f"\n{'='*70}")
    print(f"Enhanced Comparison: {name}")
    print(f"{'='*70}")
    
    results = []
    for col in ALL_FEATURES:
        if col not in original.columns or col not in synthetic.columns:
            continue
            
        orig_mean = original[col].mean()
        orig_std = original[col].std()
        orig_median = original[col].median()
        orig_skew = original[col].skew()
        
        synth_mean = synthetic[col].mean()
        synth_std = synthetic[col].std()
        synth_median = synthetic[col].median()
        synth_skew = synthetic[col].skew()
        
        mean_diff = abs(orig_mean - synth_mean) / (abs(orig_mean) + 1e-10) * 100
        std_diff = abs(orig_std - synth_std) / (abs(orig_std) + 1e-10) * 100
        median_diff = abs(orig_median - synth_median) / (abs(orig_median) + 1e-10) * 100
        
        # KS test
        ks_stat, ks_pval = ks_2samp(original[col], synthetic[col])
        
        results.append({
            'Feature': col,
            'Orig Mean': f'{orig_mean:.2f}',
            'Synth Mean': f'{synth_mean:.2f}',
            'Mean Diff %': f'{mean_diff:.1f}%',
            'Orig Std': f'{orig_std:.2f}',
            'Synth Std': f'{synth_std:.2f}',
            'Std Diff %': f'{std_diff:.1f}%',
            'KS Stat': f'{ks_stat:.3f}',
            'Quality': 'Good' if ks_stat < 0.1 else ('Fair' if ks_stat < 0.2 else 'Poor')
        })
    
    return pd.DataFrame(results)


print("Enhanced statistics comparison function defined!")

## 11. Generate Synthetic Anomaly Data

In [ ]:
# Configuration
N_SYNTHETIC = 20000  # Number of synthetic samples to generate
USE_LOG_TRANSFORM = True  # Apply log1p transform for heavy-tailed features
ENFORCE_BGP_CONSTRAINTS = True  # Enforce BGP semantic constraints

print(f"Configuration:")
print(f"  N_SYNTHETIC: {N_SYNTHETIC}")
print(f"  USE_LOG_TRANSFORM: {USE_LOG_TRANSFORM}")
print(f"  ENFORCE_BGP_CONSTRAINTS: {ENFORCE_BGP_CONSTRAINTS}")
print(f"\nOriginal high confidence anomaly samples: {len(X_anomaly)}")
print(f"\nGenerating synthetic anomaly samples...")

In [ ]:
# Method 1: Enhanced Standard SMOTE
print("\n" + "="*60)
print("Generating with Enhanced Standard SMOTE...")
print("="*60)

synthetic_smote = generate_with_smote_enhanced(
    X_anomaly, 
    N_SYNTHETIC,
    use_log_transform=USE_LOG_TRANSFORM,
    enforce_constraints=ENFORCE_BGP_CONSTRAINTS
)
print(f"Generated: {len(synthetic_smote)} samples")
synthetic_smote.head()

In [ ]:
# Method 2: Enhanced Borderline SMOTE
print("\n" + "="*60)
print("Generating with Enhanced Borderline SMOTE...")
print("="*60)

synthetic_borderline = generate_with_borderline_smote_enhanced(
    X_anomaly, 
    N_SYNTHETIC,
    use_log_transform=USE_LOG_TRANSFORM,
    enforce_constraints=ENFORCE_BGP_CONSTRAINTS,
    kind='borderline-1'
)
print(f"Generated: {len(synthetic_borderline)} samples")
synthetic_borderline.head()

In [ ]:
# Method 3: Enhanced ADASYN
print("\n" + "="*60)
print("Generating with Enhanced ADASYN...")
print("="*60)

synthetic_adasyn = generate_with_adasyn_enhanced(
    X_anomaly, 
    N_SYNTHETIC,
    use_log_transform=USE_LOG_TRANSFORM,
    enforce_constraints=ENFORCE_BGP_CONSTRAINTS
)
print(f"Generated: {len(synthetic_adasyn)} samples")
synthetic_adasyn.head()

In [ ]:
# Method 4: K-Means SMOTE
print("\n" + "="*60)
print("Generating with K-Means SMOTE...")
print("="*60)

synthetic_kmeans = generate_with_kmeans_smote_enhanced(
    X_anomaly, 
    N_SYNTHETIC,
    n_clusters=10,
    use_log_transform=USE_LOG_TRANSFORM,
    enforce_constraints=ENFORCE_BGP_CONSTRAINTS
)
print(f"Generated: {len(synthetic_kmeans)} samples")
synthetic_kmeans.head()

In [ ]:
# Method 5: Stratified SMOTE (if grouping column available)
# Try using 'Incident' as grouping column if available
synthetic_stratified = None

if 'Incident' in df_high_confidence.columns:
    print("\n" + "="*60)
    print("Generating with Stratified SMOTE (by Incident)...")
    print("="*60)
    
    synthetic_stratified = generate_stratified_smote(
        df_high_confidence,
        ALL_FEATURES,
        'Incident',
        N_SYNTHETIC,
        min_group_size=20,
        use_log_transform=USE_LOG_TRANSFORM,
        enforce_constraints=ENFORCE_BGP_CONSTRAINTS
    )
    print(f"Generated: {len(synthetic_stratified)} samples")
else:
    print("\nNo 'Incident' column found - skipping stratified SMOTE")

## 12. Validation: Verify Data Types and Constraints

In [ ]:
def validate_synthetic_data(synthetic_df, original_df, method_name):
    """
    Validate synthetic data quality.
    """
    print(f"\n{'='*60}")
    print(f"Validation: {method_name}")
    print(f"{'='*60}")
    
    issues = []
    
    # Check 1: Negative values
    for col in synthetic_df.columns:
        if (synthetic_df[col] < 0).any():
            n_negative = (synthetic_df[col] < 0).sum()
            issues.append(f"  {col}: {n_negative} negative values")
    
    if issues:
        print("Negative values found:")
        for issue in issues:
            print(issue)
    else:
        print("No negative values found")
    
    # Check 2: Integer features are integers
    non_int_issues = []
    for col in INTEGER_FEATURES:
        if col in synthetic_df.columns:
            if not (synthetic_df[col] == synthetic_df[col].astype(int)).all():
                non_int_issues.append(col)
    
    if non_int_issues:
        print(f"Non-integer values in integer features: {non_int_issues}")
    else:
        print("All integer features are properly rounded")
    
    # Check 3: BGP constraint violations
    violations = {}
    
    if 'nlri_ann' in synthetic_df.columns and 'announcements' in synthetic_df.columns:
        mask = synthetic_df['nlri_ann'] > synthetic_df['announcements']
        if mask.any():
            violations['nlri_ann > announcements'] = mask.sum()
    
    if all(col in synthetic_df.columns for col in ['origin_0', 'origin_2', 'announcements']):
        mask = (synthetic_df['origin_0'] + synthetic_df['origin_2']) > synthetic_df['announcements']
        if mask.any():
            violations['origin_0 + origin_2 > announcements'] = mask.sum()
    
    if violations:
        print(f"BGP constraint violations: {violations}")
    else:
        print("All BGP constraints satisfied")
    
    return len(issues) == 0 and len(non_int_issues) == 0 and len(violations) == 0

In [ ]:
# Validate all methods
methods = {
    'Standard SMOTE': synthetic_smote,
    'Borderline SMOTE': synthetic_borderline,
    'ADASYN': synthetic_adasyn,
    'K-Means SMOTE': synthetic_kmeans
}

if synthetic_stratified is not None:
    # For stratified, we need to exclude the grouping column
    methods['Stratified SMOTE'] = synthetic_stratified[ALL_FEATURES]

validation_results = {}
for name, synthetic in methods.items():
    validation_results[name] = validate_synthetic_data(synthetic, X_anomaly, name)

## 13. Distributional Tests

In [ ]:
# KS tests for all methods
print("\n" + "="*70)
print("Kolmogorov-Smirnov Tests for All Methods")
print("="*70)

ks_results_all = {}
for name, synthetic in methods.items():
    print(f"\n{name}:")
    ks_df = kolmogorov_smirnov_tests(X_anomaly, synthetic, ALL_FEATURES)
    ks_results_all[name] = ks_df
    
    # Summary
    n_good = (ks_df['Quality'] == 'Good').sum()
    n_fair = (ks_df['Quality'] == 'Fair').sum()
    n_poor = (ks_df['Quality'] == 'Poor').sum()
    print(f"  Quality Summary: Good={n_good}, Fair={n_fair}, Poor={n_poor}")

In [ ]:
# Correlation comparison for best method (Standard SMOTE)
print("\n" + "="*70)
print("Correlation Matrix Comparison (Standard SMOTE)")
print("="*70)

corr_results = compare_correlation_matrices(X_anomaly, synthetic_smote)
print(f"\nCorrelation preservation metrics:")
print(f"  Frobenius norm of difference: {corr_results['frobenius_norm']:.4f}")
print(f"  Max absolute difference: {corr_results['max_diff']:.4f}")
print(f"  Mean absolute difference: {corr_results['mean_diff']:.4f}")

In [ ]:
# Plot correlation comparison
fig = plot_correlation_comparison(corr_results)
plt.suptitle('Correlation Matrix Comparison: Original vs Synthetic Anomaly Data', y=1.02)
plt.show()

In [ ]:
# Plot distribution comparison
fig = plot_distribution_comparison(X_anomaly, synthetic_smote, CORE_VALIDATION_FEATURES)
plt.suptitle('Distribution Comparison: Standard SMOTE', y=1.02)
plt.show()

## 14. Enhanced Statistics Comparison

In [ ]:
# Detailed statistics comparison for each method
stats_comparison = {}

for name, synthetic in methods.items():
    stats_df = compare_statistics_enhanced(X_anomaly, synthetic, name)
    stats_comparison[name] = stats_df
    print(stats_df.to_string(index=False))

## 15. Quality Summary Across Methods

In [ ]:
# Summary comparison
print("\n" + "="*70)
print("QUALITY SUMMARY ACROSS ALL METHODS")
print("="*70)

summary_data = []
for name, ks_df in ks_results_all.items():
    avg_ks = ks_df['KS Statistic'].mean()
    n_good = (ks_df['Quality'] == 'Good').sum()
    n_fair = (ks_df['Quality'] == 'Fair').sum()
    n_poor = (ks_df['Quality'] == 'Poor').sum()
    
    # Correlation preservation for this method
    corr_result = compare_correlation_matrices(X_anomaly, methods[name])
    
    summary_data.append({
        'Method': name,
        'Avg KS Stat': f'{avg_ks:.4f}',
        'Good Features': n_good,
        'Fair Features': n_fair,
        'Poor Features': n_poor,
        'Corr Diff': f"{corr_result['mean_diff']:.4f}",
        'Validated': 'Yes' if validation_results[name] else 'No'
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

In [ ]:
# Recommend best method
print("\n" + "="*70)
print("RECOMMENDATION")
print("="*70)

# Find method with lowest average KS statistic
best_method = min(ks_results_all.items(), key=lambda x: x[1]['KS Statistic'].mean())
print(f"\nBest method based on KS statistic: {best_method[0]}")
print(f"Average KS statistic: {best_method[1]['KS Statistic'].mean():.4f}")

## 16. Save Synthetic Anomaly Data

In [ ]:
import os

# Create output directory
OUTPUT_DIR = '/home/smotaali/BGP_Traffic_Generation/results/synthetic_anomaly_data'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Output directory: {OUTPUT_DIR}")
print("\nSaving synthetic anomaly data...")

In [ ]:
# Save all methods
for name, synthetic in methods.items():
    # Create filename from method name
    filename = name.lower().replace(' ', '_').replace('-', '_') + '_anomaly.csv'
    filepath = os.path.join(OUTPUT_DIR, filename)
    synthetic.to_csv(filepath, index=False)
    print(f"Saved: {filepath} ({len(synthetic)} samples)")

In [ ]:
# Save combined dataset (original + best synthetic)
print("\nCreating combined dataset with original + synthetic data...")

# Use Standard SMOTE as default best method
best_synthetic = synthetic_smote.copy()

# Add labels
df_original_labeled = X_anomaly.copy()
df_original_labeled['data_source'] = 'original'
df_original_labeled['label'] = 'anomaly'

df_synthetic_labeled = best_synthetic.copy()
df_synthetic_labeled['data_source'] = 'synthetic'
df_synthetic_labeled['label'] = 'anomaly'

# Combine
df_combined = pd.concat([df_original_labeled, df_synthetic_labeled], ignore_index=True)

# Save combined
combined_filepath = os.path.join(OUTPUT_DIR, 'combined_anomaly_data.csv')
df_combined.to_csv(combined_filepath, index=False)
print(f"Saved combined data: {combined_filepath}")
print(f"  Original samples: {len(X_anomaly)}")
print(f"  Synthetic samples: {len(best_synthetic)}")
print(f"  Total: {len(df_combined)}")

In [ ]:
# Final summary
print("\n" + "="*70)
print("SYNTHETIC ANOMALY DATA GENERATION COMPLETE")
print("="*70)

print(f"\nInput Data:")
print(f"  Source: {DATA_PATH}")
print(f"  Confidence labels used: {HIGH_CONFIDENCE_LABELS}")
print(f"  Original high-confidence anomaly samples: {len(X_anomaly)}")

print(f"\nGeneration Settings:")
print(f"  Target synthetic samples: {N_SYNTHETIC}")
print(f"  Log transform: {USE_LOG_TRANSFORM}")
print(f"  BGP constraints: {ENFORCE_BGP_CONSTRAINTS}")

print(f"\nMethods used:")
for name in methods.keys():
    print(f"  - {name}")

print(f"\nOutput files saved to: {OUTPUT_DIR}")
for f in os.listdir(OUTPUT_DIR):
    filepath = os.path.join(OUTPUT_DIR, f)
    size = os.path.getsize(filepath) / 1024  # KB
    print(f"  - {f} ({size:.1f} KB)")

In [ ]:
# Display sample of final synthetic data
print("\nSample of generated synthetic anomaly data (Standard SMOTE):")
synthetic_smote.head(10)